# Neural DSL on AWS SageMaker

This notebook demonstrates how to use Neural DSL in AWS SageMaker environments for production-grade machine learning workflows.

## Table of Contents
1. [Setup](#setup)
2. [SageMaker Configuration](#sagemaker-config)
3. [Model Development](#model-dev)
4. [Distributed Training](#distributed)
5. [Model Deployment](#deployment)
6. [Cleanup](#cleanup)

## 1. Setup <a id='setup'></a>

In [ ]:
# Install Neural DSL
!pip install -q git+https://github.com/Lemniscate-SHA-256/Neural.git

# Install SageMaker SDK
!pip install -q sagemaker boto3

print("✓ Installation complete!")

## 2. SageMaker Configuration <a id='sagemaker-config'></a>

In [ ]:
import json

import boto3
import sagemaker

from neural.cloud.cloud_execution import CloudExecutor


# Initialize SageMaker session
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

print(f"SageMaker Role: {role}")
print(f"Region: {region}")

# Initialize Neural DSL executor
executor = CloudExecutor(timeout=1200, retry_attempts=3)

env_info = executor.get_environment_info()
print(f"\nEnvironment: {env_info['environment']}")
print(f"GPU Available: {env_info['gpu_available']}")

## 3. Model Development <a id='model-dev'></a>

In [ ]:
# Define production-ready model
production_model = """
network ProductionCNN {
    input: (224, 224, 3)
    layers:
        Conv2D(64, (3, 3), "relu", padding="same")
        BatchNormalization()
        Conv2D(64, (3, 3), "relu", padding="same")
        BatchNormalization()
        MaxPooling2D((2, 2))
        
        Conv2D(128, (3, 3), "relu", padding="same")
        BatchNormalization()
        Conv2D(128, (3, 3), "relu", padding="same")
        BatchNormalization()
        MaxPooling2D((2, 2))
        
        Conv2D(256, (3, 3), "relu", padding="same")
        BatchNormalization()
        Conv2D(256, (3, 3), "relu", padding="same")
        BatchNormalization()
        GlobalAveragePooling2D()
        
        Dense(512, "relu")
        Dropout(0.5)
        Dense(1000, "softmax")
    loss: "categorical_crossentropy"
    optimizer: Adam(learning_rate=0.001)
}
"""

# Compile model
model_path = executor.compile_model(
    production_model,
    backend='tensorflow',
    output_file='/opt/ml/model/production_model.py'
)

print(f"✓ Model compiled: {model_path}")

## 4. Distributed Training <a id='distributed'></a>

In [ ]:
# Example: Configure distributed training
distributed_config = {
    'instance_type': 'ml.p3.2xlarge',
    'instance_count': 2,
    'distribution': {
        'parameter_server': {
            'enabled': True
        }
    }
}

print("Distributed Training Configuration:")
print(json.dumps(distributed_config, indent=2))
print("\n✓ Configuration ready for SageMaker training job")

## 5. Model Deployment <a id='deployment'></a>

In [ ]:
# Deployment configuration
deployment_config = {
    'endpoint_name': 'neural-dsl-endpoint',
    'instance_type': 'ml.m5.xlarge',
    'initial_instance_count': 1,
    'model_data': model_path
}

print("Deployment Configuration:")
print(json.dumps(deployment_config, indent=2))
print("\n✓ Ready for deployment to SageMaker endpoint")

## 6. Cleanup <a id='cleanup'></a>

In [ ]:
# Cleanup resources
executor.cleanup()
print("✓ Cleanup complete!")